# Test Model TF Serving Sleep Disorder Prediction (Railway Platform)

Pada tahapan ini, kita akan melakukan percobaan prediksi dengan data yang diinputkan oleh pengguna menggunakan model yang telah dilakukan deployment pada platform Railway `https://sleep-disorder-prediction.up.railway.app/v1/models/sleep_disorder/`. Namun, tipe data untuk input tersebut harus sesuai dengan proses training model, maka dari itu setiap fitur akan dipetakan kedalam tipe data yang sama dengan proses training.

In [29]:
import tensorflow as tf
import requests
import base64
import numpy as np

# Label untuk gangguan tidur
sleep_disorder_labels = ["None", "Insomnia", "Sleep Apnea"]

# Input data
data = {
    "age": 35,
    "daily_steps": 5000,
    "heart_rate": 72,
    "physical_activity_level": 30,
    "quality_of_sleep": 7,
    "sleep_duration": 6.5,
    "stress_level": 5,
    "bmi_category": "Normal",
    "blood_pressure": "120/80",
    "gender": "Male",
    "occupation": "Software Engineer",
    "person_id": 1
}

# Convert data to tf.train.Feature
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode("utf-8")]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

features = {
    "age": _int64_feature(data["age"]),
    "daily_steps": _int64_feature(data["daily_steps"]),
    "heart_rate": _int64_feature(data["heart_rate"]),
    "physical_activity_level": _int64_feature(data["physical_activity_level"]),
    "quality_of_sleep": _int64_feature(data["quality_of_sleep"]),
    "sleep_duration": _float_feature(data["sleep_duration"]),
    "stress_level": _int64_feature(data["stress_level"]),
    "bmi_category": _bytes_feature(data["bmi_category"]),
    "blood_pressure": _bytes_feature(data["blood_pressure"]),
    "gender": _bytes_feature(data["gender"]),
    "occupation": _bytes_feature(data["occupation"]),
    "person_id": _int64_feature(data["person_id"])
}

# Create Example proto
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
serialized_example = example_proto.SerializeToString()

Setelah membuat input yang sudah dilakukan serialized agar sesuai dengan model, saatnya kita mencoba untuk memprediksi data tersebut dengan melakukan hit API `https://sleep-disorder-prediction.up.railway.app/v1/models/sleep_disorder:predict`.

In [31]:
# Prepare payload for REST API
payload = {
    "inputs": {
        "examples": [
            {"b64": base64.b64encode(serialized_example).decode("utf-8")}
        ]
    }
}

# Send POST request to the model endpoint
response = requests.post(
    "https://sleep-disorder-prediction.up.railway.app/v1/models/sleep_disorder:predict",
    json=payload
)

print(response.json())

# Get prediction result
predictions = response.json().get("outputs", [[]])[0]

# Get the predicted label
predicted_index = np.argmax(predictions)
predicted_label = sleep_disorder_labels[predicted_index]

print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {predictions}")

{'outputs': [[0.688410282, 0.0172452554, 0.154451817, 0.0174784511, 0.0518649593, 0.0447131209, 0.0258361585]]}
Predicted Label: None
Probabilities: [0.688410282, 0.0172452554, 0.154451817, 0.0174784511, 0.0518649593, 0.0447131209, 0.0258361585]
